In [31]:
from mxnet import gluon,init,nd,autograd
from mxnet.gluon import data as gdata,nn,loss as gloss
import os,sys
import d2lzh as d2l

# 1. 获取数据 / 生成iter
---

In [32]:
mnist_path = r"C:\Users\59352\.mxnet\datasets\fashion-mnist"
def load_data_fashion_mnist(batch_size,resize = None):
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)
    
    mnist_train = gdata.vision.FashionMNIST(root=mnist_path,train=True)
    mnist_test = gdata.vision.FashionMNIST(root=mnist_path,train=False)
    
    num_workers = 0 if sys.platform.startswith('win') else 4
    
    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size=batch_size,shuffle=True,num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                batch_size=batch_size,shuffle=False,num_workers=num_workers)
    return train_iter,test_iter

In [33]:
batch_size = 128
train_iter,test_iter = load_data_fashion_mnist(batch_size,resize=224)

# 2. 生成模型
---
> input: (1,1,224,224)

- network construction:(all activation are `relu`)
    ```
    1. conv 1 output : ( 1 , 96 , 54 , 54 )
    2. pool 1 output : ( 1 , 96 , 26 , 26 )
    3. conv 2 output : ( 1 , 256 , 26 , 26 )
    4. pool 2 output : ( 1 , 256 , 12 , 12 )
    
    5. conv 3 output : ( 1 , 384 , 12 , 12 ) 三个形状相同
    6. conv 4 output : ( 1 , 384 , 12 , 12 )....??????干饭层?????
    7. conv 5 output : ( 1 , 256 , 12 , 12 )
    8. pool 3 output : ( 1 , 256 , 5 , 5 )
    ```
    
    ```
    8. dense 1 output : ( 1 , 4096 )
    9. dropout 1 output : ( 1 , 4096 )
    10. dense 2 output : ( 1,4096 )
    11. dropout 2 output : ( 1 , 4096 )
    12. dense 3 output : ( 1 , 10 )
    ```

In [34]:
net = nn.Sequential()
net.add(nn.Conv2D(channels=96,kernel_size=11,strides=4,activation='relu'),
       nn.MaxPool2D(pool_size=3,strides=2),
       nn.Conv2D(channels=256,kernel_size=5,padding=2,activation='relu'),
       nn.MaxPool2D(pool_size=3,strides=2),
       
       nn.Conv2D(384,kernel_size=3,padding=1,activation='relu'),
       nn.Conv2D(384,kernel_size=3,padding=1,activation='relu'),
       nn.Conv2D(256,kernel_size=3,padding=1,activation='relu'),
       nn.MaxPool2D(pool_size=3,strides=2),
       
       nn.Dense(4096,activation='relu'),
       nn.Dropout(0.5),
       nn.Dense(4096,activation='relu'),
       nn.Dropout(0.5),
       nn.Dense(10))
net.initialize()

In [35]:
# 测试一下上面的输出形状
X = nd.random.normal(shape=(1,1,224,224))
for layer in net:
    X = layer(X)
    print(layer.name,'output shape:\t',X.shape)

conv26 output shape:	 (1, 96, 54, 54)
pool15 output shape:	 (1, 96, 26, 26)
conv27 output shape:	 (1, 256, 26, 26)
pool16 output shape:	 (1, 256, 12, 12)
conv28 output shape:	 (1, 384, 12, 12)
conv29 output shape:	 (1, 384, 12, 12)
conv30 output shape:	 (1, 256, 12, 12)
pool17 output shape:	 (1, 256, 5, 5)
dense15 output shape:	 (1, 4096)
dropout10 output shape:	 (1, 4096)
dense16 output shape:	 (1, 4096)
dropout11 output shape:	 (1, 4096)
dense17 output shape:	 (1, 10)


# 3. accuracy & train
---

In [36]:
def evaluate_accuracy(data_iter,net,ctx):
    acc_sum,n = 0.0,0
    for X,y in data_iter:
        y_hat = net(X)
        acc_sum += (y_hat.argmax(axis=1) == y.astype('float32')).sum().asscalar()
        n += y.size
    return acc_sum / n

In [37]:
def train(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs):
    
    test_acc_sum=0.0
    loss = gloss.SoftmaxCrossEntropyLoss()
    for epochs in range(num_epochs):
        train_l_sum,train_acc_sum,n=0.0,0.0,0
        for X,y in train_iter:
            X,y = X.as_in_context(ctx),y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat,y).sum()
            l.backward()
            trainer.step(batch_size)
            
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y.astype('float32')).sum().asscalar()
            n += y.size
        test_acc_sum += evaluate_accuracy(test_iter,net,ctx)
        print('epochs %d,train_l_sum %.4f,train_acc_sum %.3f,test_acc_sum %.3f' 
              %(epochs,train_l_sum/n,train_acc_sum/n,test_acc_sum))
                

# 4. 训练
---

In [38]:
lr,num_epochs,ctx = 0.01,5,d2l.try_gpu()
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
train(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

KeyboardInterrupt: 